In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.contrib.learn.python.learn.estimators.estimator import SKCompat
from tensorflow.contrib import rnn
from tensorflow.python.ops import array_ops as array_ops_
learn = tf.contrib.learn

In [ ]:
pd_ads=pd.read_csv('D:\\sale\\t_ads.csv')
pd_comment=pd.read_csv('D:\\sale\\t_comment.csv')
pd_order=pd.read_csv('D:\\sale\\t_order.csv')
pd_product=pd.read_csv('D:\\sale\\t_product.csv')
pd_sale=pd.read_csv('D:\\sale\\t_sales_sum.csv')

In [ ]:
sale_1036=pd_sale[pd_sale['shop_id']==1036]
comment_1036=pd_comment[pd_comment['shop_id']==1036]
order_1036=pd_order[pd_order['shop_id']==1036]
product_1036=pd_product[pd_product['shop_id']==1036]
ads_1036=pd_ads[pd_ads['shop_id']==1036]

sale_1036['dt']=pd.to_datetime(sale_1036['dt'])
comment_1036['create_dt']=pd.to_datetime(comment_1036['create_dt'])
order_1036['ord_dt']=pd.to_datetime(order_1036['ord_dt'])
product_1036['on_dt']=pd.to_datetime(product_1036['on_dt'])
ads_1036['create_dt']=pd.to_datetime(ads_1036['create_dt'])

sale_1036=sale_1036.sort_values(['dt'],ascending=True)
comment_1036=comment_1036.sort_values(['create_dt'],ascending=True)
order_1036=order_1036.sort_values(['ord_dt'],ascending=True)
product_1036=product_1036.sort_values(['on_dt'],ascending=True)
ads_1036=ads_1036.sort_values(['create_dt'],ascending=True)

sale_1036.to_csv('D:\\sale\\sale_1036.csv')
comment_1036.to_csv('D:\\sale\\comment_1036.csv')
order_1036.to_csv('D:\\sale\\order_1036.csv')
product_1036.to_csv('D:\\sale\\product_1036.csv')
ads_1036.to_csv('D:\\sale\\ads_1036.csv')

In [ ]:
shop_id=pd_sale['shop_id']
shop_id=list(set(shop_id))
sale_list=[]
comment_list=[]
order_list=[]
product_list=[]
ads_list=[]
sale_list=[]

for i in shop_id:
    sale_list.append(pd_sale[pd_sale['shop_id']==i])
    comment_list.append(pd_comment[pd_comment['shop_id']==i])
    order_list.append(pd_order[pd_order['shop_id']==i])
    product_list.append(pd_product[pd_product['shop_id']==i])
    ads_list.append(pd_ads[pd_ads['shop_id']==i])
    
    sale_list[i-1]=sale_list[i-1].sort_values(['dt'],ascending=True)
    comment_list[i-1]=comment_list[i-1].sort_values(['create_dt'],ascending=True)
    order_list[i-1]=order_list[i-1].sort_values(['ord_dt'],ascending=True)
    product_list[i-1]=product_list[i-1].sort_values(['on_dt'],ascending=True)
    ads_list[i-1]=ads_list[i-1].sort_values(['create_dt'],ascending=True)
    
    sale_list[i-1]['dt']=pd.to_datetime(sale_list[i-1]['dt'])
    comment_list[i-1]['create_dt']=pd.to_datetime(comment_list[i-1]['create_dt'])
    order_list[i-1]['ord_dt']=pd.to_datetime(order_list[i-1]['ord_dt'])
    product_list[i-1]['on_dt']=pd.to_datetime(product_list[i-1]['on_dt'])
    ads_list[i-1]['create_dt']=pd.to_datetime(ads_list[i-1]['create_dt'])
    print(i)

In [ ]:
for i in range(len(order_list)):
    order_list[i].drop('pid',axis=1,inplace=True)
    order_list[i].drop('shop_id',axis=1,inplace=True)
    order_list[i]=order_list[i].groupby(['ord_dt']).sum()
    
for i in range(len(comment_list)):
    comment_list[i].set_index("create_dt", inplace=True)
    comment_list[i].drop('shop_id',axis=1,inplace=True)
for i in range(len(ads_list)):
    ads_list[i].set_index("create_dt",inplace=True)
    ads_list[i].drop('shop_id',axis=1,inplace=True)
for i in range(len(sale_list)):
    sale_list[i].set_index("dt",inplace=True)
    sale_list[i].drop('shop_id',axis=1,inplace=True)

In [ ]:
date_list=[]
for i in shop_id:
    tmp=pd.DataFrame(index=pd.date_range('2016-8-1','2017-4-30'))
    date_list.append(pd.concat([tmp,comment_list[i-1], order_list[i-1],ads_list[i-1]], axis=1, join_axes=[tmp.index]))
    date_list[i-1].fillna(value=0,inplace=True)

In [ ]:
order_1036.drop('pid',axis=1,inplace=True)
order_1036=order_1036.groupby(['ord_dt']).sum()

comment_1036.drop('shop_id',axis=1,inplace=True)
order_1036.drop('shop_id',axis=1,inplace=True)
ads_1036.drop('shop_id',axis=1,inplace=True)
sale_1036.drop('shop_id',axis=1,inplace=True)


#order_1036.set_index("ord_dt", inplace=True) order_1036已将ord_dt作为索引
sale_1036.set_index("dt", inplace=True)
comment_1036.set_index("create_dt", inplace=True)
product_1036.set_index("on_dt", inplace=True)
ads_1036.set_index("create_dt", inplace=True)

sale_1036

In [ ]:
#date_list=pd.date_range('2016-6-1','2017-4-30')
date=pd.DataFrame(index=pd.date_range('2016-8-1','2017-4-30'))
data=pd.concat([date,comment_1036, order_1036,ads_1036], axis=1, join_axes=[date.index])
data.fillna(value=0,inplace=True)
data.to_csv('D:\\sale\\data_1036.csv')
data

In [ ]:
def norm(df_list):
    for i in range(len(df_list)):
        for j in [col for col in df_list[i]]:
            t_max=df_list[i][j].max()
            t_min=df_list[i][j].min()
            down = 1 if (t_max-t_min == 0) else (t_max-t_min)
            df_list[i][j]=(df_list[i][j]-t_min)/down
    pass

In [ ]:
#data.to_period('M')
data.resample('M').sum()
for i in range(len(date_list)):
    date_list[i]=date_list[i].resample('M').sum()
    
date_list[0].shape
print(np.array(date_list[0],dtype=np.float32))

norm(date_list)

for i in range(len(sale_list)):
    for j in [col for col in sale_list[i]]:
        sale_list[i][j]=pow(sale_list[i][j],0.5)
#norm(sale_list)

In [ ]:
HIDDEN_SIZE = 30
NUM_LAYERS = 2

TIMESTEPS = 30
TRAINING_STEPS = 3000
BATCH_SIZE = 32

TRAINING_EXAMPLES = 10000
TESTING_EXAMPLES = 1000
SAMPLE_GAP = 0.01

timesteps=30
num_input=14
num_hidden=18
#learning_rate= 0.01
training_steps=70000

LEARNING_RATE_BASE = 0.00001
LEARNING_RATE_DECAY = 0.99

In [ ]:
def generate_train_data(data,labels):
    X = []
    y = []
    seq=[0,31,61,92,122]

    for j in range(len(sale_list)):
        for i in range(5):
            X.append(np.array(data[j][seq[i]: seq[i] + 30],dtype=np.float32))
            y.append(np.array(labels[j].iloc[i + 2],dtype=np.float32))
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)
def generate_valid_data(data,labels):
    X = []
    y = []
    seq=[92,122]

    for j in range(len(sale_list)):
        for i in range(2):
            X.append(np.array(data[j][seq[i]: seq[i] + 30],dtype=np.float32))
            y.append(np.array(labels[j].iloc[i + 2],dtype=np.float32))
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

In [ ]:
train_X,train_y=generate_train_data(date_list,sale_list)
#valid_X,valid_y=generate_valid_data(date_list,sale_list)

In [ ]:


# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None,1])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, 1]))
}
biases = {
    'out': tf.Variable(tf.random_normal([1]))
}
global_step = tf.Variable(0, trainable=False)

In [ ]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [ ]:
with tf.variable_scope("rnn",reuse=tf.AUTO_REUSE):
    logits = RNN(X, weights, biases)
    #prediction = tf.nn.softmax(logits)
    
    # Define loss and optimizer
    #loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    #    logits=logits, labels=Y))
    #loss_op=tf.reduce_mean(tf.square(tf.reshape(logits,[-1])-tf.reshape(Y, [-1])))
    loss_op = tf.reduce_mean(tf.square(logits - Y))
    learning_rate = tf.train.exponential_decay(
            LEARNING_RATE_BASE,
            global_step,
            2000, LEARNING_RATE_DECAY,
            staircase=True)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #train_op = optimizer.minimize(loss_op,global_step=global_step)
    train_op = tf.contrib.layers.optimize_loss(
            loss_op,
            tf.contrib.framework.get_global_step(),
            optimizer='Adam',
            learning_rate=learning_rate)
    
    # Evaluate model (with test logits, for dropout to be disabled)
    #correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    correct_pred = tf.sqrt(tf.square(logits - Y))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

In [ ]:
writer = tf.summary.FileWriter("D:\\log\\jdd.log", tf.get_default_graph())
tf.summary.scalar("loss",loss_op)
merged = tf.summary.merge_all()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    batch=0
    test_X=[]
    test_y=[]
    for i in range(len(date_list)):
        test_X.append(np.array(date_list[i][153:183],dtype=np.float32))
        test_y.append(np.array(sale_list[i].iloc[-1],dtype=np.float32))
    for step in range(1, training_steps+1):
        #batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        #batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        _,summary=sess.run([train_op,merged], feed_dict={X: train_X[batch:batch+10], Y: train_y[batch:batch+10]})
        #loss=sess.run(loss_op,feed_dict={X:valid_X[batch:batch+10],Y:valid_y[batch:batch+10]})
        writer.add_summary(summary, step)
        batch+=1
        if batch==len(date_list):
            batch=0
        if step%500==0:
            # 配置运行时需要记录的信息。
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            # 运行时记录运行信息的proto。
            run_metadata = tf.RunMetadata()
            # Calculate batch loss and accuracy
            loss = sess.run(loss_op, feed_dict={X: test_X,Y: test_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) )
#            pre,rate=sess.run([logits,learning_rate],feed_dict={X:test_X,Y:test_y})
            writer.add_run_metadata(run_metadata=run_metadata, tag=("tag%d" % step), global_step=step)
#            print("rate "+ str(rate))
#            file=open('D:\\result.txt','w+')
#            result=str(pre)
#            file.writelines(result)
#            file.close()

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    #test_len = 128
    #test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    #test_label = mnist.test.labels[:test_len]
    #print("Testing Accuracy:", \
    #    sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))
    
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_X, Y: test_y}))
    pre=sess.run(logits,feed_dict={X:test_X,Y:test_y})
    predict_X=[]
    for i in range(len(date_list)):
        predict_X.append(np.array(date_list[i].iloc[243:273]))
    predict=sess.run(logits,feed_dict={X:predict_X})
    file=open('D:\\result.txt','w+')
    result=str(pre)
    file.writelines(result)
    file.close()
    writer.close()